In [31]:
import time

In [32]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## added selenium code
from selenium import webdriver
import time
from bs4 import BeautifulSoup
## For parallel execution
from multiprocessing import Pool
from tqdm.auto import tqdm
from datetime import date

today = date.today().strftime('%Y-%m-%d')

In [33]:
#Scrape data from all pages
# Enable javascript and disable automation flag
options = webdriver.ChromeOptions()
options.add_argument("--enable-javascript")
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");
options.add_argument("--disable-blink-features=AutomationControlled")
# start the instance
driver = webdriver.Chrome(options=options)

In [42]:
all_info=[]
def Properties_info(i):
            d = {}
            Parser(links[i])
            try:
                driver.find_element_by_id("_hj-wTnOw__SurveyInvitation__noThanksButton _hj-3OscV__styles__clearButton").click()
            except:
                pass
            finally:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                try:
                    address = soup.find("div",{"class":"obj-detail__address"}).get_text()
                except Exception:
                    address = "N/A"
                try:
                    linnaosa =soup.find("div",{"class":"obj-detail__area"}).get_text() 
                except Exception:
                    linnaosa ="N/A"
                for items in soup.find_all("tr"):
                    data = [item.get_text(separator=',') for item in items.find_all(['th','td'])]
                    d[data[0]]=data[-1]
                    d['aadress']=address+','+linnaosa
                    d['date'] = today
                all_info.append(d)

In [35]:
def Parser(url):
    """ Opens the URL passed to the function"""
    driver.get(url)
    return(driver)
    
def Get_Properties(pages):
    """Loops through all pages of listing, and saves their info into list."""
    all_links=[]
    for i in tqdm(range(1,pages)):
        Parser(f"https://www.city24.ee/real-estate-search/apartments-for-sale/harju-maakond/id=1-county/pg={i}")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all("article", {"class":"object-wrapper"})
        for property in results:
            link = property.find_all("a",class_="object__attributes")[0]['href']
            all_links.append("https://city24.ee"+link)
    return(all_links)

In [36]:
links = Get_Properties(54)

  0%|          | 0/53 [00:00<?, ?it/s]

In [43]:
for i in tqdm(range(0,len(links))):
    Properties_info(i)

  0%|          | 0/2650 [00:00<?, ?it/s]

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.786
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D3B8F3+2406643]
	Ordinal0 [0x00CCAF31+1945393]
	Ordinal0 [0x00BBC748+837448]
	Ordinal0 [0x00BAD9A0+776608]
	Ordinal0 [0x00BAC7C5+772037]
	Ordinal0 [0x00BACDC8+773576]
	Ordinal0 [0x00BB792F+817455]
	Ordinal0 [0x00BC256B+861547]
	Ordinal0 [0x00BC4F40+872256]
	Ordinal0 [0x00BAD096+774294]
	Ordinal0 [0x00BC23B5+861109]
	Ordinal0 [0x00C149CD+1198541]
	Ordinal0 [0x00C04096+1130646]
	Ordinal0 [0x00BDE636+976438]
	Ordinal0 [0x00BDF546+980294]
	GetHandleVerifier [0x00FA9612+2498066]
	GetHandleVerifier [0x00F9C920+2445600]
	GetHandleVerifier [0x00DD4F2A+579370]
	GetHandleVerifier [0x00DD3D36+574774]
	Ordinal0 [0x00CD1C0B+1973259]
	Ordinal0 [0x00CD6688+1992328]
	Ordinal0 [0x00CD6775+1992565]
	Ordinal0 [0x00CDF8D1+2029777]
	BaseThreadInitThunk [0x76D1FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77857A7E+286]
	RtlGetAppContainerNamedOb

In [ ]:
import pandas as pd
filename = "data/"+today+"-city24dataset.csv"
pd.DataFrame(all_info).to_csv(filename)